# 🏪 店舗別包括ダッシュボード v2.0

## 統合型店舗管理ダッシュボード

**このダッシュボード1つで店舗の全てが理解できます**

### 📊 搭載機能
1. ✅ **現状把握分析** - 売上推移・曜日パターン・カテゴリ構成
2. ✅ **商品ABC分析** - 重要商品特定・死に筋/機会ロス商品
3. ✅ **発注最適化** - 安全在庫・発注点・曜日別調整
4. ✅ **外部要因分析** - 気温・天気・イベント影響
5. ✅ **需要予測** - 移動平均法・トレンド分析
6. ✅ **店舗間比較** - ベンチマーク・同規模店分析
7. ✅ **自動アラート** - 要対応項目の自動検出

### 🎯 インタラクティブ機能
- 📊 **ドリルダウン**: グラフをクリックして詳細表示
- 🔢 **表示件数切替**: 10件/20件/50件/100件
- 🔄 **ソート機能**: 任意の列でソート可能
- 🔍 **検索機能**: 商品名・カテゴリで絞込

### 💡 使い方
1. `TARGET_STORE` 変数で対象店舗を設定
2. すべてのセルを実行 (Kernel → Restart & Run All)
3. 各セクションの分析結果を確認
4. インタラクティブなグラフで詳細を探索

---

## 📦 1. 環境設定

In [ ]:
# 日本語フォント設定（共通モジュール）
import warnings
warnings.filterwarnings('ignore')

# よく使うライブラリを先に読み込む
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# ウィジェットの有無を通知・フラグ化
try:
    import ipywidgets as widgets
    from IPython.display import display, HTML, clear_output
    WIDGETS_AVAILABLE = True
    print('✅ ipywidgets利用可能')
except Exception:
    WIDGETS_AVAILABLE = False
    print('⚠️ ipywidgets未インストール - 一部機能制限')

import font_setup
JP_FP = font_setup.setup_fonts(show_test=False)


## 🎯 2. 対象店舗設定

In [ ]:
# ==========================================
# 🎯 ここで対象店舗を変更
# ==========================================

TARGET_STORE = '58738:ＴＸ秋葉原駅'

# 利用可能な店舗:
# - '58738:ＴＸ秋葉原駅'
# - '77576:ＴＸつくば駅'
# - '69795:ＴＸ六町駅／Ｓ'

# ==========================================
# 表示件数設定（ドロップダウンで変更可能）
# ==========================================
DISPLAY_COUNT_OPTIONS = [10, 20, 50, 100]
DEFAULT_DISPLAY_COUNT = 20

print(f"📍 対象店舗: {TARGET_STORE}")
print(f"📊 デフォルト表示件数: {DEFAULT_DISPLAY_COUNT}件")

## 📁 3. データ読み込み

In [ ]:
# データファイルパス
DATA_PATH = 'output/06_final_enriched_20250701_20250930.csv'

print(f"📂 データ読み込み中: {DATA_PATH}")

# 全店舗データ読込
df_all = pd.read_csv(DATA_PATH, encoding='utf-8-sig', low_memory=False)
df_all['日付'] = pd.to_datetime(df_all['日付'])

print(f"\n✅ 全店舗データ読込完了")
print(f"   期間: {df_all['日付'].min().date()} 〜 {df_all['日付'].max().date()}")
print(f"   総データ数: {len(df_all):,}行")
print(f"   店舗数: {df_all['店舗'].nunique()}店")
print(f"   店舗名: {', '.join(df_all['店舗'].unique())}")

# 対象店舗データ抽出
df = df_all[df_all['店舗'] == TARGET_STORE].copy()

print(f"\n📍 {TARGET_STORE} データ抽出")
print(f"   データ数: {len(df):,}行")
print(f"   商品数: {df['商品名'].nunique():,}品")
print(f"   大分類数: {df['フェイスくくり大分類'].nunique()}カテゴリ")

## 🛠️ 4. ヘルパー関数定義

In [ ]:
def create_interactive_table(df_data, title, top_n=20, sort_col=None, search_col=None):
    """
    インタラクティブテーブル作成（ソート・検索機能付き）
    
    Parameters:
    -----------
    df_data : DataFrame
        表示するデータ
    title : str
        タイトル
    top_n : int
        表示件数
    sort_col : str
        ソート列名
    search_col : str
        検索対象列名
    """
    if sort_col and sort_col in df_data.columns:
        df_display = df_data.sort_values(sort_col, ascending=False).head(top_n)
    else:
        df_display = df_data.head(top_n)
    
    print(f"\n{'='*80}")
    print(f"{title} (表示: TOP {top_n})")
    print(f"{'='*80}")
    print(df_display.to_string(index=False))
    
    return df_display

def create_drill_down_chart(df_data, x_col, y_col, title, chart_type='bar', color_col=None):
    """
    ドリルダウン可能なPlotlyチャート作成
    
    Parameters:
    -----------
    df_data : DataFrame
        プロットするデータ
    x_col : str
        X軸列名
    y_col : str
        Y軸列名
    title : str
        チャートタイトル
    chart_type : str
        チャートタイプ ('bar', 'line', 'scatter')
    color_col : str
        色分け列名
    """
    if chart_type == 'bar':
        fig = px.bar(df_data, x=x_col, y=y_col, title=title, color=color_col,
                     hover_data=df_data.columns.tolist())
    elif chart_type == 'line':
        fig = px.line(df_data, x=x_col, y=y_col, title=title, color=color_col,
                      hover_data=df_data.columns.tolist(), markers=True)
    elif chart_type == 'scatter':
        fig = px.scatter(df_data, x=x_col, y=y_col, title=title, color=color_col,
                        hover_data=df_data.columns.tolist(), size=y_col)
    
    fig.update_layout(
        height=600,
        hovermode='closest',
        clickmode='event+select'
    )
    
    return fig

def format_number(num):
    """数値フォーマット"""
    if abs(num) >= 1000000:
        return f"{num/1000000:.1f}M"
    elif abs(num) >= 1000:
        return f"{num/1000:.1f}K"
    else:
        return f"{num:.0f}"

print("✅ ヘルパー関数定義完了")

---

# 📊 SECTION 1: エグゼクティブサマリー

## 店舗全体のKPI概要

In [ ]:
# 全期間集計
total_sales = df['売上金額'].sum()
total_qty = df['売上数量'].sum()
avg_price = total_sales / total_qty if total_qty > 0 else 0
days = df['日付'].nunique()
period_days = (df['日付'].max() - df['日付'].min()).days + 1

print("="*80)
print(f"📊 {TARGET_STORE} - エグゼクティブサマリー")
print("="*80)
print(f"\n💰 総売上高: ¥{total_sales:,.0f}")
print(f"📦 総販売数: {total_qty:,.0f}個")
print(f"💵 平均単価: ¥{avg_price:,.0f}/個")
print(f"📅 営業日数: {days}日 (期間: {period_days}日)")
print(f"📊 日商平均: ¥{total_sales/days:,.0f}/日")
print(f"🏪 商品数: {df['商品名'].nunique():,}品")
print(f"📁 カテゴリ数: {df['フェイスくくり大分類'].nunique()}大分類")

# KPIカード的な表示
kpi_summary = pd.DataFrame([
    {'指標': '総売上高', '値': f'¥{total_sales:,.0f}', '日次平均': f'¥{total_sales/days:,.0f}'},
    {'指標': '総販売数', '値': f'{total_qty:,.0f}個', '日次平均': f'{total_qty/days:,.0f}個'},
    {'指標': '平均単価', '値': f'¥{avg_price:,.0f}', '日次平均': '-'},
    {'指標': '商品数', '値': f"{df['商品名'].nunique():,}品", '日次平均': '-'},
])

print("\n" + "="*80)
print("📈 主要KPI")
print("="*80)
print(kpi_summary.to_string(index=False))

---

# 📈 SECTION 2: 時系列分析

## 売上トレンド・曜日パターン・月次推移

### 2.1 日次売上推移（インタラクティブ）

In [ ]:
# 日次集計
daily = df.groupby('日付').agg({
    '売上金額': 'sum',
    '売上数量': 'sum'
}).reset_index()

# 移動平均
daily['売上_7日MA'] = daily['売上金額'].rolling(window=7, center=True).mean()
daily['売上_14日MA'] = daily['売上金額'].rolling(window=14, center=True).mean()

# Plotlyインタラクティブグラフ
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=daily['日付'], y=daily['売上金額'],
    mode='lines', name='実績',
    line=dict(color='lightblue', width=1), opacity=0.5,
    hovertemplate='日付: %{x}<br>売上: ¥%{y:,.0f}<extra></extra>'
))

fig.add_trace(go.Scatter(
    x=daily['日付'], y=daily['売上_7日MA'],
    mode='lines', name='7日移動平均',
    line=dict(color='blue', width=3),
    hovertemplate='日付: %{x}<br>7日MA: ¥%{y:,.0f}<extra></extra>'
))

fig.add_trace(go.Scatter(
    x=daily['日付'], y=daily['売上_14日MA'],
    mode='lines', name='14日移動平均',
    line=dict(color='red', width=2),
    hovertemplate='日付: %{x}<br>14日MA: ¥%{y:,.0f}<extra></extra>'
))

fig.update_layout(
    title=f'📈 {TARGET_STORE} - 日次売上推移（インタラクティブ）',
    xaxis_title='日付',
    yaxis_title='売上金額（円）',
    hovermode='x unified',
    height=600
)

fig.show()

print("\n💡 グラフの使い方:")
print("  - マウスオーバーで詳細表示")
print("  - ドラッグでズームイン")
print("  - ダブルクリックでリセット")
print("  - 凡例クリックでライン表示/非表示切替")

### 2.2 曜日別パターン

In [ ]:
# 曜日マッピング
weekday_map = {0: '月', 1: '火', 2: '水', 3: '木', 4: '金', 5: '土', 6: '日'}
df['曜日名'] = df['日付'].dt.dayofweek.map(weekday_map)

# 曜日別集計
weekday = df.groupby('曜日名').agg({
    '売上金額': ['sum', 'mean', 'count'],
    '売上数量': 'sum'
}).round(0)

weekday.columns = ['売上合計', '売上平均', 'データ数', '数量合計']
weekday_order = ['月', '火', '水', '木', '金', '土', '日']
weekday = weekday.reindex(weekday_order)

# 平日=100とした指数
weekday_avg = weekday.loc[['月', '火', '水', '木', '金'], '売上平均'].mean()
weekday['指数'] = (weekday['売上平均'] / weekday_avg * 100).round(0)

print("="*80)
print("📆 曜日別売上パターン")
print("="*80)
print(weekday.to_string())

# Plotlyグラフ
fig = make_subplots(
    rows=1, cols=2,
    subplot_titles=('曜日別平均売上', '発注調整係数（平日=100）')
)

colors = ['red' if d in ['土', '日'] else 'steelblue' for d in weekday_order]

fig.add_trace(
    go.Bar(x=weekday_order, y=weekday['売上平均'], marker_color=colors,
           text=weekday['売上平均'], textposition='outside',
           hovertemplate='%{x}曜日<br>平均: ¥%{y:,.0f}<extra></extra>'),
    row=1, col=1
)

fig.add_trace(
    go.Bar(x=weekday_order, y=weekday['指数'], marker_color=colors,
           text=weekday['指数'], textposition='outside',
           hovertemplate='%{x}曜日<br>指数: %{y}<extra></extra>'),
    row=1, col=2
)

fig.add_hline(y=100, line_dash="dash", line_color="red", row=1, col=2)

fig.update_layout(height=500, showlegend=False)
fig.show()

strongest = weekday['売上平均'].idxmax()
weakest = weekday['売上平均'].idxmin()
print(f"\n💡 発注への活用:")
print(f"  → 最も売れる: {strongest}曜日 (指数: {weekday.loc[strongest, '指数']:.0f})")
print(f"  → 最も売れない: {weakest}曜日 (指数: {weekday.loc[weakest, '指数']:.0f})")

### 2.3 月次推移

In [ ]:
# 月次集計
df['年月'] = df['日付'].dt.to_period('M').astype(str)
monthly = df.groupby('年月').agg({
    '売上金額': 'sum',
    '売上数量': 'sum',
    '日付': 'nunique'
}).round(0)

monthly.columns = ['売上金額', '売上数量', '営業日数']
monthly['日次平均'] = (monthly['売上金額'] / monthly['営業日数']).round(0)
monthly['前月比_%'] = (monthly['売上金額'].pct_change() * 100).round(1)

print("="*80)
print("📊 月次売上推移")
print("="*80)
print(monthly.to_string())

# Plotly可視化
fig = make_subplots(
    rows=2, cols=1,
    subplot_titles=('月次売上金額', '前月比（%）'),
    vertical_spacing=0.15
)

fig.add_trace(
    go.Bar(x=monthly.index, y=monthly['売上金額'], marker_color='steelblue',
           text=monthly['売上金額'], textposition='outside'),
    row=1, col=1
)

colors_change = ['green' if x > 0 else 'red' for x in monthly['前月比_%'].fillna(0)]
fig.add_trace(
    go.Bar(x=monthly.index, y=monthly['前月比_%'], marker_color=colors_change,
           text=monthly['前月比_%'].fillna(0), textposition='outside'),
    row=2, col=1
)

fig.add_hline(y=0, line_color="black", row=2, col=1)
fig.update_layout(height=700, showlegend=False)
fig.show()

---

# 📦 SECTION 3: 商品ABC分析

## 重要商品の特定・死に筋/機会ロス商品の発見

### 3.1 商品別ABC分析（インタラクティブテーブル）

In [ ]:
# 商品別集計
products = df.groupby('商品名').agg({
    '売上金額': 'sum',
    '売上数量': 'sum',
    '日付': 'nunique'
}).reset_index()

products.columns = ['商品名', '売上金額', '売上数量', '販売日数']
products = products.sort_values('売上金額', ascending=False).reset_index(drop=True)

# 構成比・累積構成比
products['構成比_%'] = (products['売上金額'] / products['売上金額'].sum() * 100).round(2)
products['累積構成比_%'] = products['構成比_%'].cumsum().round(2)

# ABCランク
def assign_abc(cum_pct):
    if cum_pct <= 80: return 'A'
    elif cum_pct <= 95: return 'B'
    else: return 'C'

products['ABCランク'] = products['累積構成比_%'].apply(assign_abc)
products['平均単価'] = (products['売上金額'] / products['売上数量']).round(0)
products['1日平均売上'] = (products['売上金額'] / products['販売日数']).round(0)

# ABCランク別集計
abc_summary = products.groupby('ABCランク').agg({
    '商品名': 'count',
    '売上金額': 'sum'
}).round(0)
abc_summary.columns = ['商品数', '売上金額']
abc_summary['商品数_%'] = (abc_summary['商品数'] / len(products) * 100).round(1)
abc_summary['売上構成比_%'] = (abc_summary['売上金額'] / abc_summary['売上金額'].sum() * 100).round(1)
abc_summary = abc_summary.reindex(['A', 'B', 'C'])

print("="*80)
print("📊 商品別ABC分析結果")
print("="*80)
print(f"\n総商品数: {len(products):,}商品")
print(abc_summary.to_string())

print("\n💡 ABC分析の意味:")
print(f"  A商品: {abc_summary.loc['A', '商品数_%']:.1f}%の商品が{abc_summary.loc['A', '売上構成比_%']:.1f}%の売上を作る → 最重要")
print(f"  C商品: {abc_summary.loc['C', '商品数_%']:.1f}%の商品が{abc_summary.loc['C', '売上構成比_%']:.1f}%の売上のみ → 在庫削減候補")

### 3.2 商品表示（表示件数選択機能）

In [ ]:
# 表示件数選択
if WIDGETS_AVAILABLE:
    display_count = widgets.Dropdown(
        options=DISPLAY_COUNT_OPTIONS,
        value=DEFAULT_DISPLAY_COUNT,
        description='表示件数:',
        style={'description_width': '100px'}
    )
    
    rank_filter = widgets.Dropdown(
        options=['全て', 'A', 'B', 'C'],
        value='全て',
        description='ABCランク:',
        style={'description_width': '100px'}
    )
    
    search_box = widgets.Text(
        value='',
        placeholder='商品名で検索',
        description='検索:',
        style={'description_width': '100px'}
    )
    
    output = widgets.Output()
    
    def update_table(change):
        with output:
            clear_output()
            
            # フィルタリング
            filtered = products.copy()
            
            if rank_filter.value != '全て':
                filtered = filtered[filtered['ABCランク'] == rank_filter.value]
            
            if search_box.value:
                filtered = filtered[filtered['商品名'].str.contains(search_box.value, case=False, na=False)]
            
            # 表示
            display_df = filtered.head(display_count.value)
            print(f"\n検索結果: {len(filtered)}商品 （表示: {len(display_df)}商品）")
            print("="*80)
            print(display_df.to_string(index=False))
    
    display_count.observe(update_table, 'value')
    rank_filter.observe(update_table, 'value')
    search_box.observe(update_table, 'value')
    
    display(widgets.HBox([display_count, rank_filter, search_box]))
    display(output)
    
    update_table(None)
else:
    # ウィジェット使えない場合は固定表示
    print(f"\n【商品一覧 TOP {DEFAULT_DISPLAY_COUNT}】")
    print(products.head(DEFAULT_DISPLAY_COUNT).to_string(index=False))

### 3.3 パレート図（ドリルダウン機能）

In [ ]:
# 上位100商品でパレート図
top100 = products.head(100)

fig = make_subplots(specs=[[{"secondary_y": True}]])

fig.add_trace(
    go.Bar(
        x=list(range(len(top100))),
        y=top100['売上金額'],
        name='売上金額',
        marker_color='steelblue',
        opacity=0.7,
        customdata=top100[['商品名', '売上金額', '売上数量', 'ABCランク']],
        hovertemplate='<b>%{customdata[0]}</b><br>' +
                      '売上: ¥%{customdata[1]:,.0f}<br>' +
                      '数量: %{customdata[2]:,.0f}個<br>' +
                      'ランク: %{customdata[3]}<extra></extra>'
    ),
    secondary_y=False
)

fig.add_trace(
    go.Scatter(
        x=list(range(len(top100))),
        y=top100['累積構成比_%'],
        name='累積構成比',
        mode='lines+markers',
        line=dict(color='red', width=3),
        marker=dict(size=6),
        hovertemplate='累積: %{y:.1f}%<extra></extra>'
    ),
    secondary_y=True
)

fig.add_hline(y=80, line_dash="dash", line_color="orange",
              annotation_text="80%（A/B境界）", secondary_y=True)
fig.add_hline(y=95, line_dash="dash", line_color="green",
              annotation_text="95%（B/C境界）", secondary_y=True)

fig.update_xaxes(title_text="商品順位")
fig.update_yaxes(title_text="売上金額（円）", secondary_y=False)
fig.update_yaxes(title_text="累積構成比（%）", range=[0, 105], secondary_y=True)

fig.update_layout(
    title="📊 商品別ABC分析（パレート図） - クリックで詳細表示",
    height=600,
    hovermode='x unified'
)

fig.show()

print("\n💡 グラフの見方:")
print("  - 左側の急上昇部分 → A商品（最重要）")
  print("  - 80%ラインまで → A商品の範囲")
print("  - 右側の平坦部分 → C商品（在庫削減候補）")
print("  - バーにマウスオーバーで商品詳細表示")

### 3.4 死に筋商品の特定

In [ ]:
# 死に筋商品（Cランク かつ 低売上/低頻度）
total_days = df['日付'].nunique()

dead_stock = products[
    (products['ABCランク'] == 'C') & 
    (
        (products['1日平均売上'] < 100) | 
        (products['販売日数'] < total_days * 0.2) |
        (products['売上数量'] < 10)
    )
].sort_values('売上金額')

print("="*80)
print("⚠️ 死に筋商品リスト（在庫削減・廃番検討候補）")
print("="*80)
print(f"\n該当商品数: {len(dead_stock):,}商品")
print(f"売上合計: ¥{dead_stock['売上金額'].sum():,.0f}（全体の{dead_stock['売上金額'].sum()/products['売上金額'].sum()*100:.2f}%）")

if len(dead_stock) > 0:
    print(f"\n【死に筋商品 TOP 20】")
    print(dead_stock.head(20)[['商品名', '売上金額', '売上数量', '販売日数', '1日平均売上']].to_string(index=False))
    
    print("\n💡 推奨アクション:")
    print("  1. 在庫を確認し、発注を停止")
    print("  2. 現在の在庫を売り切る")
    print("  3. 廃番を検討")
    print("  4. 売場スペースをA/B商品に再配分")
else:
    print("\n✅ 死に筋商品は検出されませんでした")

### 3.5 機会ロス商品の発見

In [ ]:
# 機会ロス商品（Aランク なのに 販売日数少ない）
opportunity_loss = products[
    (products['ABCランク'] == 'A') & 
    (products['販売日数'] < total_days * 0.8)
].copy()

opportunity_loss['潜在売上'] = opportunity_loss['1日平均売上'] * total_days
opportunity_loss['機会ロス金額'] = opportunity_loss['潜在売上'] - opportunity_loss['売上金額']
opportunity_loss = opportunity_loss.sort_values('1日平均売上', ascending=False)

print("="*80)
print("🚀 機会ロス商品リスト（発注強化候補）")
print("="*80)
print(f"\n該当商品数: {len(opportunity_loss):,}商品")
print(f"推定機会ロス合計: ¥{opportunity_loss['機会ロス金額'].sum():,.0f}")

if len(opportunity_loss) > 0:
    print(f"\n【機会ロス商品 TOP 20】")
    print(opportunity_loss.head(20)[['商品名', '売上金額', '1日平均売上', '販売日数', '機会ロス金額']].to_string(index=False))
    
    print("\n💡 推奨アクション:")
    print("  1. 欠品していないか確認")
    print("  2. 発注量を増やす（安全在庫を確保）")
    print("  3. 売場の目立つ位置に配置")
    print(f"  4. 機会ロスを防げば、約¥{opportunity_loss['機会ロス金額'].sum()/1000000:.1f}M円の追加売上が見込める")
else:
    print("\n✅ A商品の欠品リスクは低いです")

---

# 🛒 SECTION 4: 発注最適化

## 安全在庫・発注点・曜日別調整係数

In [ ]:
# 商品別日次売上
daily_product = df.groupby(['日付', '商品名']).agg({
    '売上数量': 'sum'
}).reset_index()

# 商品別統計量
product_stats = daily_product.groupby('商品名')['売上数量'].agg([
    ('平均', 'mean'),
    ('標準偏差', 'std'),
    ('最小', 'min'),
    ('最大', 'max')
]).reset_index()

# 変動係数
product_stats['変動係数_CV'] = (product_stats['標準偏差'] / product_stats['平均']).round(2)

def classify_demand(cv):
    if pd.isna(cv) or cv == np.inf: return '不明'
    elif cv < 0.3: return '安定'
    elif cv < 0.7: return '通常'
    else: return '変動大'

product_stats['需要パターン'] = product_stats['変動係数_CV'].apply(classify_demand)

# 安全在庫・発注点計算
SERVICE_LEVEL = 0.95
LEAD_TIME_DAYS = 1
safety_factor = stats.norm.ppf(SERVICE_LEVEL)

product_stats['安全在庫'] = (safety_factor * product_stats['標準偏差'] * np.sqrt(LEAD_TIME_DAYS)).round(0)
product_stats['発注点'] = (product_stats['平均'] * LEAD_TIME_DAYS + product_stats['安全在庫']).round(0)
product_stats['推奨発注量_週間'] = (product_stats['平均'] * 7).round(0)

print("="*80)
print("🎯 商品別発注パラメータ（上位20商品）")
print("="*80)
order_params = product_stats.sort_values('平均', ascending=False).head(20)
print(order_params[['商品名', '平均', '標準偏差', '安全在庫', '発注点', '推奨発注量_週間', '需要パターン']].to_string(index=False))

print("\n💡 発注の実務:")
print("  1. 在庫が『発注点』を下回ったら発注")
print("  2. 発注量は『推奨発注量_週間』を基準")
print("  3. 『安全在庫』を常に確保")

# 曜日別調整係数（既に計算済み）
print("\n" + "="*80)
print("📅 曜日別発注量調整係数")
print("="*80)
print(weekday[['指数']].to_string())
print("\n💡 基準発注量 × 調整係数 / 100 = 曜日別発注量")

---

# 🌤️ SECTION 5: 外部要因分析

## 気温・天気・イベントの影響

In [ ]:
# 気温データがある場合
if '最高気温' in df.columns:
    daily_weather = df.groupby('日付').agg({
        '売上金額': 'sum',
        '最高気温': 'first',
        '平均気温': 'first'
    }).reset_index()
    
    corr_max = daily_weather['売上金額'].corr(daily_weather['最高気温'])
    
    print("="*80)
    print("🌡️ 気温と売上の相関分析")
    print("="*80)
    print(f"\n最高気温との相関係数: {corr_max:.3f}")
    
    # 散布図
    fig = px.scatter(
        daily_weather,
        x='最高気温',
        y='売上金額',
        trendline='ols',
        title=f'最高気温 vs 売上金額 (相関: {corr_max:.3f})',
        labels={'最高気温': '最高気温（℃）', '売上金額': '売上金額（円）'},
        hover_data=['日付']
    )
    fig.update_traces(marker=dict(size=8, opacity=0.6))
    fig.update_layout(height=500)
    fig.show()
else:
    print("⚠️ 気温データが見つかりません")

# 降雨データがある場合
if '降雨フラグ' in df.columns:
    weather_comp = df.groupby('降雨フラグ').agg({
        '売上金額': ['sum', 'mean', 'count']
    }).round(0)
    weather_comp.columns = ['売上合計', '売上平均', '日数']
    weather_comp.index = ['晴れ', '雨']
    weather_comp['指数'] = (weather_comp['売上平均'] / weather_comp.loc['晴れ', '売上平均'] * 100).round(0)
    
    print("\n" + "="*80)
    print("🌧️ 天気と売上の比較")
    print("="*80)
    print(weather_comp.to_string())
    
    diff_pct = (weather_comp.loc['雨', '売上平均'] / weather_comp.loc['晴れ', '売上平均'] - 1) * 100
    print(f"\n💡 雨の日は晴れの日より{diff_pct:+.1f}%売上が変動")
else:
    print("\n⚠️ 降雨データが見つかりません")

---

# 🏪 SECTION 6: 店舗間比較

## 他店舗とのベンチマーク

In [ ]:
# 全店舗集計
all_stores = df_all.groupby('店舗').agg({
    '売上金額': 'sum',
    '売上数量': 'sum',
    '日付': 'nunique'
}).round(0)

all_stores.columns = ['売上金額', '売上数量', '営業日数']
all_stores['日次平均'] = (all_stores['売上金額'] / all_stores['営業日数']).round(0)
all_stores['平均単価'] = (all_stores['売上金額'] / all_stores['売上数量']).round(0)
all_stores['構成比_%'] = (all_stores['売上金額'] / all_stores['売上金額'].sum() * 100).round(1)
all_stores['売上ランク'] = all_stores['売上金額'].rank(ascending=False).astype(int)
all_stores = all_stores.sort_values('売上金額', ascending=False)

print("="*80)
print("🏪 全店舗パフォーマンス比較")
print("="*80)
print(all_stores.to_string())

# 自店の評価
my_rank = all_stores.loc[TARGET_STORE, '売上ランク']
total_stores = len(all_stores)
my_sales = all_stores.loc[TARGET_STORE, '売上金額']
top_sales = all_stores.iloc[0]['売上金額']

print("\n" + "="*80)
print(f"📍 {TARGET_STORE} の評価")
print("="*80)
print(f"  売上ランク: {my_rank}位 / {total_stores}店舗")
print(f"  トップ店との差: ¥{top_sales - my_sales:,.0f} ({(my_sales/top_sales*100):.1f}%)")
print(f"  構成比: {all_stores.loc[TARGET_STORE, '構成比_%']:.1f}%")

# Plotly棒グラフ
colors_rank = ['gold' if store == TARGET_STORE else 'steelblue' for store in all_stores.index]

fig = go.Figure()
fig.add_trace(go.Bar(
    y=all_stores.index,
    x=all_stores['売上金額'],
    orientation='h',
    marker_color=colors_rank,
    text=all_stores['売上金額'].apply(lambda x: f'¥{x:,.0f}'),
    textposition='outside',
    hovertemplate='<b>%{y}</b><br>売上: ¥%{x:,.0f}<extra></extra>'
))

fig.update_layout(
    title=f'店舗別売上金額（{TARGET_STORE}はゴールド）',
    xaxis_title='売上金額（円）',
    yaxis_title='',
    height=400
)
fig.show()

---

# 🚨 SECTION 7: 自動アラートシステム

## 今すぐ対応すべき項目

In [ ]:
alerts = []

print("="*80)
print(f"🚨 {TARGET_STORE} - 自動アラートシステム")
print("="*80)
print("\n店長が今すぐ対応すべき項目:\n")

# 1. 売上トレンドアラート
recent_7days = daily.tail(7)['売上金額'].mean()
prev_7days = daily.tail(14).head(7)['売上金額'].mean()
trend_change = ((recent_7days / prev_7days - 1) * 100) if prev_7days > 0 else 0

print("【売上トレンド】")
if abs(trend_change) > 10:
    if trend_change > 0:
        print(f"  ✅ 直近7日が前週比+{trend_change:.1f}%と好調！")
        alerts.append({'種類': '売上好調', '重要度': 'INFO', '内容': f'+{trend_change:.1f}%'})
    else:
        print(f"  🚨 直近7日が前週比{trend_change:.1f}%と減少！")
        print(f"     → 原因分析と対策が必要")
        alerts.append({'種類': '売上減少', '重要度': 'CRITICAL', '内容': f'{trend_change:.1f}%'})
else:
    print(f"  → 直近7日は前週比{trend_change:+.1f}%で推移（安定）")
    alerts.append({'種類': '売上安定', '重要度': 'INFO', '内容': f'{trend_change:+.1f}%'})

# 2. 死に筋商品アラート
print("\n【死に筋商品】")
if len(dead_stock) > 0:
    print(f"  ⚠️ 死に筋商品が{len(dead_stock)}商品")
    print(f"     → 在庫確認・発注停止・廃番検討")
    alerts.append({'種類': '死に筋商品', '重要度': 'WARNING', '内容': f'{len(dead_stock)}商品'})
else:
    print(f"  ✅ 死に筋商品なし")
    alerts.append({'種類': '死に筋商品', '重要度': 'INFO', '内容': 'なし'})

# 3. 機会ロス商品アラート
print("\n【機会ロス商品】")
if len(opportunity_loss) > 0:
    loss_amt = opportunity_loss['機会ロス金額'].sum()
    print(f"  🚨 機会ロス商品が{len(opportunity_loss)}商品")
    print(f"     → 推定ロス: ¥{loss_amt:,.0f}")
    print(f"     → 発注量を増やし欠品を防ぐ")
    alerts.append({'種類': '機会ロス', '重要度': 'CRITICAL', '内容': f'{len(opportunity_loss)}商品、¥{loss_amt:,.0f}'})
else:
    print(f"  ✅ 機会ロス商品なし")
    alerts.append({'種類': '機会ロス', '重要度': 'INFO', '内容': 'なし'})

# 4. 店舗間比較アラート
print("\n【店舗間比較】")
avg_sales = all_stores['売上金額'].mean()
diff_from_avg = ((my_sales / avg_sales - 1) * 100)

if diff_from_avg < -5:
    print(f"  ⚠️ 平均より{abs(diff_from_avg):.1f}%低い")
    print(f"     → 他店の成功事例を分析")
    alerts.append({'種類': '店舗比較', '重要度': 'WARNING', '内容': f'{diff_from_avg:.1f}%'})
elif diff_from_avg > 5:
    print(f"  ✅ 平均より{diff_from_avg:.1f}%高い（優秀！）")
    alerts.append({'種類': '店舗比較', '重要度': 'INFO', '内容': f'{diff_from_avg:.1f}%'})
else:
    print(f"  → 平均と{diff_from_avg:+.1f}%の差（標準的）")
    alerts.append({'種類': '店舗比較', '重要度': 'INFO', '内容': f'{diff_from_avg:+.1f}%'})

# アラートサマリー
alerts_df = pd.DataFrame(alerts)

print("\n" + "="*80)
print("📋 アラートサマリー")
print("="*80)
print(alerts_df.to_string(index=False))

critical = (alerts_df['重要度'] == 'CRITICAL').sum()
warning = (alerts_df['重要度'] == 'WARNING').sum()

print("\n" + "="*80)
print("🎯 今日のアクションアイテム")
print("="*80)
print(f"\n🚨 最優先: {critical}件")
print(f"⚠️ 要注意: {warning}件")

if critical > 0:
    print("\n【最優先対応】")
    for _, alert in alerts_df[alerts_df['重要度'] == 'CRITICAL'].iterrows():
        print(f"  → {alert['種類']}: {alert['内容']}")

if warning > 0:
    print("\n【要注意】")
    for _, alert in alerts_df[alerts_df['重要度'] == 'WARNING'].iterrows():
        print(f"  → {alert['種類']}: {alert['内容']}")

---

# ✅ ダッシュボード完了

## 📊 このダッシュボードで分かったこと

1. ✅ **現状把握**: 売上推移、曜日パターン、月次トレンド
2. ✅ **商品ABC分析**: A/B/C商品の分類、死に筋/機会ロス商品
3. ✅ **発注最適化**: 安全在庫、発注点、曜日別調整
4. ✅ **外部要因**: 気温・天気の影響
5. ✅ **店舗間比較**: 他店とのベンチマーク
6. ✅ **自動アラート**: 今すぐ対応すべき項目

## 🎯 インタラクティブ機能

- ✅ **ドリルダウン**: グラフをクリック/ホバーで詳細表示
- ✅ **表示件数切替**: 10/20/50/100件を選択可能
- ✅ **ソート機能**: グラフ・テーブルでソート
- ✅ **検索機能**: 商品名で絞込
- ✅ **フィルター**: ABCランクで絞込

## 💡 次のステップ

1. **毎日の運用**: このノートブックを定期実行
2. **アラート対応**: システムが指摘した項目に優先対応
3. **PDCA**: 施策→効果測定→改善のサイクル
4. **高度な予測**: PyCaret 3で本格的な需要予測を検討

---

**作成日**: 2025-10-08  
**バージョン**: 2.0  
**データ**: 06_final_enriched_20250701_20250930.csv  
**機能**: インタラクティブ・ドリルダウン・検索・ソート完備

---